In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1648937572890_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Check schema of source table in glue catalog and number of rows

In [2]:
flights_dynamic = glueContext.create_dynamic_frame.from_catalog(database="flights_summary", table_name="delays")
init_rowcount = flights_dynamic.count()
print("\n Count: " + str(init_rowcount))
flights_dynamic.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 Count: 1048575
root
|-- 'year': long
|-- 'month': long
|-- 'dayofmonth': long
|-- 'dayofweek': long
|-- 'deptime': long
|-- 'crsdeptime': long
|-- 'arrtime': long
|-- 'crsarrtime': long
|-- 'uniquecarrier': string
|-- 'flightnum': long
|-- 'tailnum': string
|-- 'actualelapsedtime': long
|-- 'crselapsedtime': long
|-- 'airtime': long
|-- 'arrdelay': long
|-- 'depdelay': long
|-- 'origin': string
|-- 'dest': string
|-- 'distance': long
|-- 'taxiin': long
|-- 'taxiout': long
|-- 'cancelled': long
|-- 'cancellationcode': string
|-- 'diverted': long
|-- 'carrierdelay': long
|-- 'weatherdelay': long
|-- 'nasdelay': long
|-- 'securitydelay': long
|-- 'lateaircraftdelay': long

#### input source Amazon S3

In [3]:
s3_raw_flights_ddf = glueContext.create_dynamic_frame.from_catalog(
    database="flights_summary",
    table_name="delays",
    transformation_ctx="s3_raw_flights_csv",
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### node dropfields transform

we can also confirm the fields have been dropped by printing schema using output dynamic df 

In [5]:

# Script generated for node Drop Fields
dropfields_ddf = DropFields.apply(
    frame=s3_raw_flights_ddf,
    paths=[
        "'year'",
        "'deptime'",
        "'dayofweek'",
        "'crsdeptime'",
        "'arrtime'",
        "'crsarrtime'",
        "'flightnum'",
        "'tailnum'",
        "'crselapsedtime'",
        "'airtime'",
        "'arrdelay'",
        "'depdelay'",
        "'taxiin'",
        "'taxiout'",
        "'cancelled'",
        "'cancellationcode'",
    ],
    transformation_ctx="dropfields",
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
dropfields_ddf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- 'month': long
|-- 'dayofmonth': long
|-- 'uniquecarrier': string
|-- 'actualelapsedtime': long
|-- 'origin': string
|-- 'dest': string
|-- 'distance': long
|-- 'diverted': long
|-- 'carrierdelay': long
|-- 'weatherdelay': long
|-- 'nasdelay': long
|-- 'securitydelay': long
|-- 'lateaircraftdelay': long

#### Map cols to new names and cast types

In [7]:

# Script generated for node Apply Mapping
apply_mapping_ddf = ApplyMapping.apply(
    frame=dropfields_ddf,
    mappings=[
        ("'month'", "long", "'month'", "int"),
        ("'dayofmonth'", "long", "'dayofmonth'", "int"),
        ("'uniquecarrier'", "string", "'uniquecarrier'", "string"),
        ("'actualelapsedtime'", "long", "'actualelapsedtime'", "long"),
        ("'origin'", "string", "'origin'", "string"),
        ("'dest'", "string", "'dest'", "string"),
        ("'distance'", "long", "'distance'", "long"),
        ("'diverted'", "long", "'diverted'", "tinyint"),
        ("'carrierdelay'", "long", "'carrierdelay'", "long"),
        ("'weatherdelay'", "long", "'weatherdelay'", "long"),
        ("'nasdelay'", "long", "'nasdelay'", "long"),
        ("'securitydelay'", "long", "'securitydelay'", "long"),
        ("'lateaircraftdelay'", "long", "'lateaircraftdelay'", "long"),
    ],
    transformation_ctx="col_rename_and_cast_types",
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
apply_mapping_ddf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- 'month': int
|-- 'dayofmonth': int
|-- 'uniquecarrier': string
|-- 'actualelapsedtime': long
|-- 'origin': string
|-- 'dest': string
|-- 'distance': long
|-- 'diverted': byte
|-- 'carrierdelay': long
|-- 'weatherdelay': long
|-- 'nasdelay': long
|-- 'securitydelay': long
|-- 'lateaircraftdelay': long

#### drop dupicates transform and count number of rows dropped

In [9]:

# Script generated for node Drop Duplicates
dropduplicates_ddf = DynamicFrame.fromDF(
    apply_mapping_ddf.toDF().dropDuplicates(),
    glueContext,
    "DropDuplicates_node",
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
final_rowcount = dropduplicates_ddf.toDF().count()
print(f" \n Number of duplicates dropped:{init_rowcount- final_rowcount}") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 
 Number of duplicates dropped:6782

#### create target node S3 sink and define parition cols and parquet format

Also set table name in catalog to crawl output  

In [11]:

target_s3_node = glueContext.getSink(
    path="s3://flight-delays-2008/output_glue_etl/",
    connection_type="s3",
    updateBehavior="UPDATE_IN_DATABASE",
    partitionKeys=["'month'"],
    compression="gzip",
    enableUpdateCatalog=True,
    transformation_ctx="target_s3_node",
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
target_s3_node.setCatalogInfo(
    catalogDatabase="flights_summary", catalogTableName="fl_delays_with_codes"
)
target_s3_node.setFormat("glueparquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### write to s3  and check tables/data created in catalog

In [16]:
target_s3_node.writeFrame(dropduplicates_ddf)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
spark.sql('use flights_summary')
spark.sql('show tables').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+-----------+
|       database|           tableName|isTemporary|
+---------------+--------------------+-----------+
|flights_summary|              delays|      false|
|flights_summary|fl_delays_with_codes|      false|
+---------------+--------------------+-----------+

In [62]:
spark.sql('select * from fl_delays_with_codes limit 10').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---------------+-------------------+--------+------+----------+----------+--------------+--------------+----------+---------------+-------------------+-------+
|'dayofmonth'|'uniquecarrier'|'actualelapsedtime'|'origin'|'dest'|'distance'|'diverted'|'carrierdelay'|'weatherdelay'|'nasdelay'|'securitydelay'|'lateaircraftdelay'|'month'|
+------------+---------------+-------------------+--------+------+----------+----------+--------------+--------------+----------+---------------+-------------------+-------+
|           3|             WN|                 64|     HOU|   HRL|       276|         0|             0|             0|         4|              0|                 61|      3|
|           3|             WN|                107|     MDW|   LIT|       544|         0|            11|             0|         2|              0|                 43|      3|
|           3|             WN|                 58|     OKC|   DAL|       181|         0|             3|             0|         8| 